In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd

In [7]:
# Configurar el WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Si quieres ver el navegador, elimina esta línea

# Instalar y configurar el driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL de la página
url = "https://www.adamchoi.co.uk/overs/detailed"
driver.get(url)

# Esperar a que la página cargue
time.sleep(5)

# Verificar si la página se abrió correctamente
if "adamchoi" in driver.current_url:
    print("✅ Página cargada con éxito")
else:
    print("❌ Error al cargar la página")

# Cerrar el navegador
driver.quit()



✅ Página cargada con éxito


In [11]:
# Reabrir el navegador
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
time.sleep(3)

# Intentar seleccionar Colombia en el dropdown
try:
    country_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@id='country']"))
    )
    country_dropdown.click()
    
    colombia_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//option[contains(text(), 'Colombia')]"))
    )
    colombia_option.click()
    
    print("✅ Filtro de Colombia aplicado correctamente")
    time.sleep(2)
except Exception as e:
    print(f"❌ Error al seleccionar Colombia: {e}")
    

# Filtrar por "All Matches"
try:
    all_matches_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'All matches')]"))
    )
    all_matches_button.click()
    print("✅ Filtro de 'All Matches' aplicado correctamente")
    time.sleep(2)  # Esperamos a que se actualicen los datos
except Exception as e:
    print(f"❌ Error al seleccionar 'All Matches': {e}")


# Cerrar el navegador
driver.quit()


✅ Filtro de Colombia aplicado correctamente
✅ Filtro de 'All Matches' aplicado correctamente


In [35]:
# Reabrir el navegador
driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
time.sleep(3)

# Intentar seleccionar Colombia en el dropdown
try:
    country_dropdown = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//select[@id='country']"))
    )
    country_dropdown.click()
    
    colombia_option = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//option[contains(text(), 'Colombia')]"))
    )
    colombia_option.click()
    
    print("✅ Filtro de Colombia aplicado correctamente")
    time.sleep(2)
except Exception as e:
    print(f"❌ Error al seleccionar Colombia: {e}")
    

# Filtrar por "All Matches"
try:
    all_matches_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//label[contains(text(), 'All matches')]"))
    )
    all_matches_button.click()
    print("✅ Filtro de 'All Matches' aplicado correctamente")
    time.sleep(2)  # Esperamos a que se actualicen los datos
except Exception as e:
    print(f"❌ Error al seleccionar 'All Matches': {e}")
    

# Esperar a que la tabla de partidos esté cargada completamente
try:
    print("Esperando a que la tabla de partidos cargue...")
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )
    print("✅ Tabla de partidos detectada, extrayendo datos...")
except Exception as e:
    print(f"❌ Error: La tabla no se cargó correctamente. {e}")
    driver.quit()

data = []

for year in range(2025, 2022, -1):  # Iterar sobre los años disponibles (2025, 2024, 2023)
    try:
        # Seleccionar el año en el dropdown
        option = driver.find_element(By.XPATH, f"//select[@id='season']/option[@label='{year}']")
        option.click()
        print(f"⌛ Cambiando a datos del año {year}...")

        # Esperar hasta que la tabla se actualice con los nuevos datos
        WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.XPATH, "//table//tr[1]/td[1]"), str(year))
        )
        print(f"✅ Datos del año {year} cargados correctamente.")

        # Extraer las filas de la tabla después de asegurarse de que el contenido cambió
        matches = driver.find_elements(By.XPATH, "//tr")

        for match in matches:
            cols = match.find_elements(By.TAG_NAME, "td")

            if len(cols) >= 6:  # Verificamos que la fila tenga al menos 6 columnas
                fecha = cols[0].text.strip()  # Fecha del partido
                tarjeta_roja_local = cols[1].text.strip() if cols[1].text.strip() else "0"  # Tarjetas rojas equipo local
                equipo_local = cols[2].text.strip()  # Nombre del equipo local
                marcador = cols[3].text.strip()  # Marcador del partido
                equipo_visitante = cols[4].text.strip()  # Nombre del equipo visitante
                tarjeta_roja_visitante = cols[5].text.strip() if cols[5].text.strip() else "0"  # Tarjetas rojas equipo visitante

                # Guardar los datos en la lista
                data.append([fecha, equipo_local, marcador, equipo_visitante, tarjeta_roja_local, tarjeta_roja_visitante])

                # Imprimir valores para depuración
                print(f"Año {year} - Fecha: {fecha}, Local: {equipo_local}, Marcador: {marcador}, Visitante: {equipo_visitante}, TR Local: {tarjeta_roja_local}, TR Visitante: {tarjeta_roja_visitante}")

    except Exception as e:
        print(f"❌ Error al seleccionar el año {year}: {e}")
        continue  # Si falla con un año, pasa al siguiente

# Cerrar el navegador
driver.quit()

# Convertir a DataFrame
df = pd.DataFrame(data, columns=["Fecha", "Equipo Local", "Marcador", "Equipo Visitante", "Tarjeta Roja Local", "Tarjeta Roja Visitante"])


# Mostrar los primeros resultados
print(df.head())

✅ Filtro de Colombia aplicado correctamente
✅ Filtro de 'All Matches' aplicado correctamente
Esperando a que la tabla de partidos cargue...
✅ Tabla de partidos detectada, extrayendo datos...
⌛ Cambiando a datos del año 2025...
✅ Datos del año 2025 cargados correctamente.
Año 2025 - Fecha: 26-01-2025, Local: Alianza Petrolera, Marcador: 1 - 1, Visitante: Independiente Medellín, TR Local: 0, TR Visitante: 0
Año 2025 - Fecha: 02-02-2025, Local: Deportivo Pereira, Marcador: 0 - 0, Visitante: Alianza Petrolera, TR Local: 0, TR Visitante: 0
Año 2025 - Fecha: 07-02-2025, Local: Deportes Tolima, Marcador: 3 - 1, Visitante: Alianza Petrolera, TR Local: 0, TR Visitante: 0
Año 2025 - Fecha: 17-02-2025, Local: Atlético Bucaramanga, Marcador: 0 - 0, Visitante: Alianza Petrolera, TR Local: 0, TR Visitante: 0
Año 2025 - Fecha: 22-02-2025, Local: Alianza Petrolera, Marcador: 3 - 2, Visitante: Atlético Nacional, TR Local: 0, TR Visitante: 0
Año 2025 - Fecha: 01-03-2025, Local: Once Caldas, Marcador: 1 

In [36]:
# Guardar en un archivo CSV
df.to_csv("partidos_colombia_2024.csv", index=False, encoding="utf-8")

print("✅ Archivo CSV guardado exitosamente.")


✅ Archivo CSV guardado exitosamente.
